In [1]:
import geopandas as gpd
import pandas as pd
import os,re,json
import numpy as np
from p_tqdm import p_map

In [ ]:

# pip install mbutil

In [84]:
# lets generate the overview pictures 
#
floc = "tmp/county.geojson"
gpd.read_file('data/CountiesUA.geojson')[['OBJECTID', 'CTYUA20CD', 'CTYUA20NM','LONG', 'LAT', 'geometry']].to_file(floc, driver="GeoJSON")

os.system('tippecanoe -Z0 -z5 -f -o tiles/countries.mbtiles --coalesce-densest-as-needed --generate-id '+ floc)


For layer 0, using name "county"
216 features, 1304369 bytes of geometry, 16 bytes of separate metadata, 10662 bytes of string pool
  97.1%  5/15/10  


0

In [85]:
base  = gpd.read_file('data/OA2011.geojson')[['OA11CD', 'LAD11CD',
       'geometry']]
centroids = gpd.read_file('data/centroids2011.geojson')[['lsoa01cd', 'geometry']]

base.to_file("tmp/OA11.geojson", driver="GeoJSON")
centroids.to_file("tmp/centroids11.geojson", driver="GeoJSON")

os.system('tippecanoe -Z6 -z12 -f -o tiles/area_centroids_6_9.mbtiles --coalesce-densest-as-needed --generate-id ./tmp/OA%d.geojson ./tmp/centroids%d.geojson'%(11,11))

# tippecanoe -o tiles/test.mbtiles -Z9 -z10 -f ./data/tile_OA11.geojson ./data/tile_centroids11.geojson


# gdf_base = gdf_base.append(df,ignore_index=True)
# gdf_base .to_file("myfile.gpkg", layer="layer1", driver="GPKG")

For layer 0, using name "OA11"
For layer 1, using name "centroids11"
215786 features, 22300915 bytes of geometry, 1960 bytes of separate metadata, 2377501 bytes of string pool
tile 6/32/21 size is 554829 with detail 12, >500000    
Going to try keeping the sparsest 81.11% of the features to make it fit
tile 6/32/21 size is 532122 with detail 12, >500000    
Going to try keeping the sparsest 68.59% of the features to make it fit
tile 6/31/20 size is 1660791 with detail 12, >500000    
Going to try keeping the sparsest 27.10% of the features to make it fit
tile 6/32/21 size is 501192 with detail 12, >500000    
Going to try keeping the sparsest 61.58% of the features to make it fit
tile 6/31/21 size is 1736401 with detail 12, >500000    
Going to try keeping the sparsest 25.92% of the features to make it fit
tile 6/31/20 size is 951030 with detail 12, >500000    
Going to try keeping the sparsest 12.82% of the features to make it fit
tile 6/31/21 size is 1021684 with detail 12, >500000  

0

In [ ]:
# base  = gpd.read_file('data/OA2011.geojson')[['OA01CD', 'LAD01CD',
#        'geometry']]
# centroids = gpd.read_file('data/centroids2011.geojson')[['lsoa01cd', 'geometry']]

# base.to_file("tmp/OA11.geojson", driver="GeoJSON")
# centroids.to_file("tmp/centroids11.geojson", driver="GeoJSON")

# os.system('tippecanoe -Z6 -z9 -f -o tiles/area_centroids_6_9.mbtiles --coalesce-densest-as-needed --generate-id ./tmp/OA%d.geojson ./tmp/centroids%d.geojson'%(11,11))




In [86]:
# merge
os.system('tile-join -o ./tiles/merged.mbtiles -f ./tiles/countries.mbtiles ./tiles/area_centroids_6_9.mbtiles')

0

In [87]:
# tippecanoe -z3 -o countries-z3.mbtiles --coalesce-densest-as-needed ne_10m_admin_0_countries.geojson
# tippecanoe -zg -Z4 -o states-Z4.mbtiles --coalesce-densest-as-needed --extend-zooms-if-still-dropping ne_10m_admin_1_states_provinces.geojson
# tile-join -o states-countries.mbtiles countries-z3.mbtiles states-Z4.mbtiles

In [89]:
## convert to static server

os.system('rm -rf ./tiles/tileserver; mb-util --image_format=pbf ./tiles/merged.mbtiles ./tiles/tileserver')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % gzip -d -r -S .pbf %')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % mv % %.pbf')


DEBUG:mbutil.util:Exporting MBTiles to disk
DEBUG:mbutil.util:./tiles/merged.mbtiles --> ./tiles/tileserver
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:4 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:5 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:6 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:7 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:8 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:9 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:10 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:11 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:12 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:13 / 6470 tiles exported
DEBU

INFO:mbutil.util:726 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:727 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:728 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:729 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:730 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:731 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:732 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:733 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:734 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:735 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:736 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:737 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:738 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:739 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:740

INFO:mbutil.util:1464 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1465 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1466 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1467 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1468 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1469 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1470 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1471 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1472 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1473 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1474 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1475 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1476 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1477 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:m

DEBUG:mbutil.util:flipping
INFO:mbutil.util:2209 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2210 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2211 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2212 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2213 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2214 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2215 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2216 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2217 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2218 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2219 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2220 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2221 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2222 / 6470 tiles exported
DEBUG:

INFO:mbutil.util:3126 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3127 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3128 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3129 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3130 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3131 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3132 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3133 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3134 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3135 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3136 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3137 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3138 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3139 / 6470 tiles exported
DEBUG:mbutil.util:flipping
INFO:m

...

mv ./12/1999/1374.pbf ./12/1999/1374.pbf.pbf
mv ./12/1999/1360.pbf ./12/1999/1360.pbf.pbf
mv ./12/1999/1376.pbf ./12/1999/1376.pbf.pbf
mv ./12/1999/1377.pbf ./12/1999/1377.pbf.pbf
mv ./12/1999/1352.pbf ./12/1999/1352.pbf.pbf
mv ./12/1999/1353.pbf ./12/1999/1353.pbf.pbf
mv ./12/1999/1379.pbf ./12/1999/1379.pbf.pbf
mv ./12/1999/1351.pbf ./12/1999/1351.pbf.pbf
mv ./12/1999/1350.pbf ./12/1999/1350.pbf.pbf
mv ./12/1999/1378.pbf ./12/1999/1378.pbf.pbf
mv ./12/1999/1354.pbf ./12/1999/1354.pbf.pbf
mv ./12/1999/1368.pbf ./12/1999/1368.pbf.pbf
mv ./12/1999/1383.pbf ./12/1999/1383.pbf.pbf
mv ./12/1999/1382.pbf ./12/1999/1382.pbf.pbf
mv ./12/1999/1369.pbf ./12/1999/1369.pbf.pbf
mv ./12/1999/1355.pbf ./12/1999/1355.pbf.pbf
mv ./12/1999/1357.pbf ./12/1999/1357.pbf.pbf
mv ./12/1999/1380.pbf ./12/1999/1380.pbf.pbf
mv ./12/1999/1381.pbf ./12/1999/1381.pbf.pbf
mv ./12/1999/1356.pbf ./12/1999/1356.pbf.pbf
mv ./12/1999/1331.pbf ./12/1999/1331.pbf.pbf
mv ./12/1999/1330.pbf ./12/1999/1330.pbf.pbf
mv ./12/19

mv ./12/2060/1348.pbf ./12/2060/1348.pbf.pbf
mv ./12/2060/1352.pbf ./12/2060/1352.pbf.pbf
mv ./12/2060/1346.pbf ./12/2060/1346.pbf.pbf
mv ./12/2060/1347.pbf ./12/2060/1347.pbf.pbf
mv ./12/2060/1353.pbf ./12/2060/1353.pbf.pbf
mv ./12/2060/1345.pbf ./12/2060/1345.pbf.pbf
mv ./12/2060/1351.pbf ./12/2060/1351.pbf.pbf
mv ./12/2060/1350.pbf ./12/2060/1350.pbf.pbf
mv ./12/2060/1344.pbf ./12/2060/1344.pbf.pbf
mv ./12/2060/1340.pbf ./12/2060/1340.pbf.pbf
mv ./12/2060/1354.pbf ./12/2060/1354.pbf.pbf
mv ./12/2060/1368.pbf ./12/2060/1368.pbf.pbf
mv ./12/2060/1369.pbf ./12/2060/1369.pbf.pbf
mv ./12/2060/1355.pbf ./12/2060/1355.pbf.pbf
mv ./12/2060/1341.pbf ./12/2060/1341.pbf.pbf
mv ./12/2060/1357.pbf ./12/2060/1357.pbf.pbf
mv ./12/2060/1343.pbf ./12/2060/1343.pbf.pbf
mv ./12/2060/1342.pbf ./12/2060/1342.pbf.pbf
mv ./12/2060/1356.pbf ./12/2060/1356.pbf.pbf
mv ./12/2060/1337.pbf ./12/2060/1337.pbf.pbf
mv ./12/2060/1336.pbf ./12/2060/1336.pbf.pbf
mv ./12/2060/1334.pbf ./12/2060/1334.pbf.pbf
mv ./12/20

mv ./12/2002/1345.pbf ./12/2002/1345.pbf.pbf
mv ./12/2002/1351.pbf ./12/2002/1351.pbf.pbf
mv ./12/2002/1350.pbf ./12/2002/1350.pbf.pbf
mv ./12/2002/1344.pbf ./12/2002/1344.pbf.pbf
mv ./12/2002/1378.pbf ./12/2002/1378.pbf.pbf
mv ./12/2002/1340.pbf ./12/2002/1340.pbf.pbf
mv ./12/2002/1354.pbf ./12/2002/1354.pbf.pbf
mv ./12/2002/1368.pbf ./12/2002/1368.pbf.pbf
mv ./12/2002/1383.pbf ./12/2002/1383.pbf.pbf
mv ./12/2002/1382.pbf ./12/2002/1382.pbf.pbf
mv ./12/2002/1369.pbf ./12/2002/1369.pbf.pbf
mv ./12/2002/1355.pbf ./12/2002/1355.pbf.pbf
mv ./12/2002/1341.pbf ./12/2002/1341.pbf.pbf
mv ./12/2002/1357.pbf ./12/2002/1357.pbf.pbf
mv ./12/2002/1343.pbf ./12/2002/1343.pbf.pbf
mv ./12/2002/1380.pbf ./12/2002/1380.pbf.pbf
mv ./12/2002/1381.pbf ./12/2002/1381.pbf.pbf
mv ./12/2002/1342.pbf ./12/2002/1342.pbf.pbf
mv ./12/2002/1356.pbf ./12/2002/1356.pbf.pbf
mv ./12/2002/1331.pbf ./12/2002/1331.pbf.pbf
mv ./12/2002/1330.pbf ./12/2002/1330.pbf.pbf
mv ./12/2002/1332.pbf ./12/2002/1332.pbf.pbf
mv ./12/20

mv ./12/2004/1383.pbf ./12/2004/1383.pbf.pbf
mv ./12/2004/1382.pbf ./12/2004/1382.pbf.pbf
mv ./12/2004/1369.pbf ./12/2004/1369.pbf.pbf
mv ./12/2004/1355.pbf ./12/2004/1355.pbf.pbf
mv ./12/2004/1341.pbf ./12/2004/1341.pbf.pbf
mv ./12/2004/1357.pbf ./12/2004/1357.pbf.pbf
mv ./12/2004/1343.pbf ./12/2004/1343.pbf.pbf
mv ./12/2004/1380.pbf ./12/2004/1380.pbf.pbf
mv ./12/2004/1381.pbf ./12/2004/1381.pbf.pbf
mv ./12/2004/1342.pbf ./12/2004/1342.pbf.pbf
mv ./12/2004/1356.pbf ./12/2004/1356.pbf.pbf
mv ./12/2004/1331.pbf ./12/2004/1331.pbf.pbf
mv ./12/2004/1330.pbf ./12/2004/1330.pbf.pbf
mv ./12/2004/1332.pbf ./12/2004/1332.pbf.pbf
mv ./12/2004/1333.pbf ./12/2004/1333.pbf.pbf
mv ./12/2004/1327.pbf ./12/2004/1327.pbf.pbf
mv ./12/2004/1337.pbf ./12/2004/1337.pbf.pbf
mv ./12/2004/1336.pbf ./12/2004/1336.pbf.pbf
mv ./12/2004/1334.pbf ./12/2004/1334.pbf.pbf
mv ./12/2004/1335.pbf ./12/2004/1335.pbf.pbf
mv ./12/2032/1338.pbf ./12/2032/1338.pbf.pbf
mv ./12/2032/1310.pbf ./12/2032/1310.pbf.pbf
mv ./12/20

0